In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from collections import defaultdict

from model.Base import Base
from model.Location import Location
from model.User import User
from model.user_devices import user_devices

from sqlalchemy import create_engine, text, func
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import NullPool

from IPython.display import display

DB='postgresql+psycopg2:///ucnstudy'

engine = create_engine(DB, echo=False, poolclass=NullPool)
Base.metadata.bind = engine
Session = sessionmaker(bind=engine)

In [2]:
ses = Session()
users = ses.query(User)

for user in users:
    sql_user_devices = text('select * from user, user_devices where user_devices.user_id =:user').bindparams(user = user.id)
    user_devices = ses.execute(sql_user_devices)

    #will get the starting time and ending times considering all devices
    print ("user: " + str(user.id) + "=======================")
    for dev in user_devices:
        sql_beg_day = text('SELECT distinct devid, locations.entertime, name \
        FROM locations join\
        (SELECT DATE(entertime) as date_entered, MIN(entertime) as min_time\
        FROM locations\
        WHERE devid = :d_id and extract (hour from entertime) > 3\
        GROUP BY date(entertime))\
        AS grp ON grp.min_time = locations.entertime order by locations.entertime;').bindparams(d_id = dev.device_id)
        result_beg_day = ses.execute(sql_beg_day)

        sql_end_day = text('SELECT distinct devid, locations.exittime, name \
        FROM locations join\
        (SELECT DATE(exittime) as date_entered, MIN(exittime) as min_time\
        FROM locations\
        WHERE devid = :d_id\
        GROUP BY date(exittime))\
        AS grp ON grp.min_time = locations.exittime order by locations.exittime;').bindparams(d_id = dev.device_id)
        result_end_day = ses.execute(sql_end_day)

        sql_entertime = text('SELECT distinct devid, locations.entertime, name \
        FROM locations join\
        (SELECT DATE(entertime) as date_entered, MIN(entertime) as min_time\
        FROM locations\
        WHERE devid = :d_id \
        GROUP BY date(entertime))\
        AS grp ON grp.min_time = locations.entertime order by locations.entertime;').bindparams(d_id = dev.device_id)
            
        #sql_entertime = text('select devid, entertime from locations where devid =:d_id').bindparams(d_id = dev.device_id)
            
        result_entertime = ses.execute(sql_entertime)

        info_end = defaultdict(list)
        for row in result_end_day:
            info_end['devid'].append(str(row[0]))
            info_end['end'].append(row[1])
            info_end['location'].append(row[2])
        
        info_beg = defaultdict(list)
        for row in result_beg_day:
            info_beg['devid'].append(row[0])
            info_beg['beg'].append(row[1])
            info_beg['location'].append(row[2])
            
        #add days that only have value before 3 am
        for row in result_entertime:
            timst = row[1]
            in_list = False
            for dt in info_beg['beg']:
                if dt.day == timst.day and dt.month == timst.month and dt.year == timst.year:
                    in_list = True
            if in_list == False:
                info_beg['beg'].append(timst)
                info_beg['devid'].append(row[0])
                info_beg['location'].append(row[2])

        df_beg = pd.DataFrame(info_beg)
        display(df_beg)
        df_end = pd.DataFrame(info_end)
        display(df_end)

user: 1=======================


,beg,devid,location
0,2015-03-05 14:17:27.902,6,"Rue Quincampoix, Paris"
1,2015-03-06 13:06:22.797,6,"Rue Quincampoix, Paris"
2,2015-03-07 10:27:24.482,6,"Rue Quincampoix, Paris"
3,2015-03-08 12:59:12.644,6,"Rue Quincampoix, Paris"
4,2015-03-09 17:45:03.785,6,"Place d'Italie, Paris"
5,2015-03-10 16:30:04.230,6,"Rue Quincampoix, Paris"
6,2015-03-11 10:47:10.600,6,"Rue Quincampoix, Paris"
7,2015-03-12 09:05:45.481,6,"Rue des Prêtres Saint-Germain l'Auxerrois, Paris"
8,2015-03-14 12:09:50.591,6,"Rue Quincampoix, Paris"
9,2015-03-15 13:16:11.127,6,"Rue Quincampoix, Paris"


,devid,end,location
0,6,2015-03-05 14:19:43.903,"Rue Quincampoix, Paris"
1,6,2015-03-06 00:43:49.135,"Rue Quincampoix, Paris"
2,6,2015-03-07 15:13:22.112,"Rue Quincampoix, Paris"
3,6,2015-03-08 00:01:26.500,"Rue Quincampoix, Paris"
4,6,2015-03-09 19:48:38.972,"Place d'Italie, Paris"
5,6,2015-03-10 21:57:47.168,"Rue Quincampoix, Paris"
6,6,2015-03-11 17:54:00.313,"Rue Quincampoix, Paris"
7,6,2015-03-12 10:34:34.788,"Rue des Prêtres Saint-Germain l'Auxerrois, Paris"
8,6,2015-03-14 12:19:31.711,"Rue Quincampoix, Paris"
9,6,2015-03-15 14:29:16.381,"Rue Quincampoix, Paris"


user: 2=======================


,beg,devid,location
0,2015-02-25 15:04:49.324,1,"Rue Quincampoix, Paris"
1,2015-02-27 09:36:04.320,1,"Rue Martre, Clichy"
2,2015-03-04 20:25:10.951,1,"Rue des Prêtres Saint-Germain l'Auxerrois, Paris"
3,2015-03-05 09:31:14.955,1,"Rue Quincampoix, Paris"
4,2015-03-06 13:22:07.192,1,"Rue Quincampoix, Paris"
5,2015-03-07 09:18:57.352,1,"Rue Thiers, Boulogne-Billancourt"
6,2015-03-17 19:12:03.785,1,"Place de l'Hôtel de ville, Nanterre"
7,2015-03-18 08:13:35.000,1,"Rue de Paris, Clichy"
8,2015-03-19 04:48:34.555,1,"Avenue du Maréchal Foch, Chatou"
9,2015-03-20 08:29:27.000,1,"Rue de Paris, Clichy"


...

,beg,devid,location
0,2015-11-24 15:58:22,47,"Main Street, Awsworth"
1,2015-11-25 11:28:39,47,"Mansfield Road, Nottingham"
2,2015-11-26 11:23:30,47,"Firbeck Road, Nottingham"
3,2015-11-27 19:15:26,47,"Albert Street, Ilkeston"
4,2015-11-28 13:41:47,47,"Rutland Street, Ilkeston"
5,2015-11-29 12:47:49,47,"Redfield Way, Nottingham"
6,2015-11-30 11:45:18,47,"Cat and Fiddle Lane, United Kingdom"
7,2015-12-01 12:05:40,47,"Cat and Fiddle Lane, United Kingdom"
8,2015-12-02 05:31:46,47,"Douglas Avenue, Awsworth"
9,2015-12-03 11:39:44,47,"Firbeck Road, Nottingham"


,devid,end,location
0,47,2015-11-24 21:50:28,"Main Street, Awsworth"
1,47,2015-11-25 11:04:46,"Main Street, Awsworth"
2,47,2015-11-26 11:12:17,"Main Street, Awsworth"
3,47,2015-11-27 19:05:50,"Main Street, Awsworth"
4,47,2015-11-28 13:35:35,"Main Street, Awsworth"
5,47,2015-11-29 12:25:24,"Douglas Avenue, Awsworth"
6,47,2015-11-30 11:29:55,"Main Street, Awsworth"
7,47,2015-12-01 11:50:24,"Douglas Avenue, Awsworth"
8,47,2015-12-02 00:56:20,"Awsworth Lane, Kimberley"
9,47,2015-12-03 11:20:37,"Douglas Avenue, Awsworth"


user: 26=======================


,beg,devid,location
0,2015-11-24 15:15:02.942,45,"Storey's Gate, London"
1,2015-11-25 18:22:20.771,45,"Storey's Gate, London"


,devid,end,location
0,45,2015-11-24 15:20:02.942,"Storey's Gate, London"
1,45,2015-11-25 18:27:20.771,"Storey's Gate, London"
